In [16]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData, PrimaryKeyConstraint
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship

In [2]:
found_csv= "Negin/found_list.csv"
found_df= pd.read_csv(found_csv)
found_df.head()

,Unnamed: 0,ID,Sex,Age,Weight,Color,Breed,Intake Jurisdiction,Intake Date,link
0,1,A1571111,F,2 Months,19.5 LBS,WHITE,AIREDALE TERR MIX,TUSTIN,4/15/2018,http://petadoption.ocpetinfo.com/rescuetrack/s...
1,2,A1571195,M,2 Years,50.4 LBS,WHITE,ALASKAN HUSKY,CYPRESS,4/16/2018,http://petadoption.ocpetinfo.com/rescuetrack/s...
2,3,A1559633,S,2 Years,50.0 LBS,BLUE,AM PIT BULL TER,SAN JUAN CAP,4/17/2018,http://petadoption.ocpetinfo.com/rescuetrack/s...
3,4,A1569453,N,6 Years,60.0 LBS,BLUE MERLE,AUST CATTLE DOG,HUNTINGTON BCH,4/4/2018,http://petadoption.ocpetinfo.com/rescuetrack/s...
4,5,A1571387,N,5 Years,24.2 LBS,TRICOLOR BLACK,BEAGLE,FULLERTON,4/17/2018,http://petadoption.ocpetinfo.com/rescuetrack/s...


In [3]:
adoption_df= found_df.rename(columns={"ID":"Pet_ID", "Intake Jurisdiction": "Intake_Jurisdiction", "Intake Date":"Intake_Date"})
adoption_df= adoption_df[["Pet_ID","Sex","Age","Weight","Color","Breed","Intake_Jurisdiction", "Intake_Date", "link"]]
adoption_df= adoption_df.fillna(0)
adoption_df.head()

,Pet_ID,Sex,Age,Weight,Color,Breed,Intake_Jurisdiction,Intake_Date,link
0,A1571111,F,2 Months,19.5 LBS,WHITE,AIREDALE TERR MIX,TUSTIN,4/15/2018,http://petadoption.ocpetinfo.com/rescuetrack/s...
1,A1571195,M,2 Years,50.4 LBS,WHITE,ALASKAN HUSKY,CYPRESS,4/16/2018,http://petadoption.ocpetinfo.com/rescuetrack/s...
2,A1559633,S,2 Years,50.0 LBS,BLUE,AM PIT BULL TER,SAN JUAN CAP,4/17/2018,http://petadoption.ocpetinfo.com/rescuetrack/s...
3,A1569453,N,6 Years,60.0 LBS,BLUE MERLE,AUST CATTLE DOG,HUNTINGTON BCH,4/4/2018,http://petadoption.ocpetinfo.com/rescuetrack/s...
4,A1571387,N,5 Years,24.2 LBS,TRICOLOR BLACK,BEAGLE,FULLERTON,4/17/2018,http://petadoption.ocpetinfo.com/rescuetrack/s...


In [4]:
#adoption_df.to_csv("Holly/adoptable_dogs.csv",encoding="utf-8", index=False, header=True )

In [5]:
engine = create_engine("sqlite:///dog_recommender.sqlite")

In [6]:
Base=declarative_base()

In [7]:
class Dog_breeds(Base):
    
    __tablename__ = 'dog_breed'
    extend_existing=True
    id = Column(Integer, primary_key=True, autoincrement=True)
    Dog_Breed= Column(String(50))
    Adapts_Well_to_Apartment_Living= Column(Integer)
    Affectionate_with_Family= Column(Integer)
    Amount_Of_Shedding= Column(Integer)
    Dog_Friendly= Column(Integer)
    Drooling_Potential= Column(Integer)
    Easy_To_Groom= Column(Integer)
    Easy_To_Train= Column(Integer)
    Energy_Level= Column(Integer)
    Exercise_Needs= Column(Integer)
    Friendly_Toward_Strangers= Column(Integer)
    General_Health= Column(Integer)
    Good_For_Novice_Owners= Column(Integer)
    Incredibly_Kid_Friendly_Dogs= Column(Integer)
    Intelligence= Column(Integer)
    Intensity= Column(Integer)
    Potential_For_Mouthiness= Column(Integer)
    Potential_For_Playfulness= Column(Integer)
    Potential_For_Weight_Gain= Column(Integer)
    Prey_Drive= Column(Integer)
    Sensitivity_Level= Column(Integer)
    Size= Column(Integer)
    Tendency_To_Bark_Or_Howl= Column(Integer)
    Tolerates_Being_Alone= Column(Integer)
    Tolerates_Cold_Weather= Column(Integer)
    Tolerates_Hot_Weather= Column(Integer)
    Wanderlust_Potential= Column(Integer)

    
    def __repr__(self):
        return f"id={self.id}, name={self.Dog_Breed}"

In [8]:
class Lost_Dogs(Base):
    
    __tablename__ = 'lost_dogs'
    id = Column(Integer, primary_key=True)
    Pet_ID= Column(String(20))
    Sex= Column(String(20))
    Age= Column(Integer)
    Weight= Column(Integer)
    Color= Column(String)
    Breed= Column(String)
    Intake_Jurisdiction= Column(String)
    Intake_Date= Column(String)


        
    def __repr__(self):
        return f"id={self.id}, name={self.Pet_ID}"

In [9]:
class Adoptable_Dogs(Base):
    
    __tablename__ = 'adoptable_dogs'
    id = Column(Integer, primary_key=True)
    Pet_ID= Column(String(20))
    Sex= Column(String(20))
    Age= Column(Integer)
    Weight= Column(Integer)
    Color= Column(String)
    Breed= Column(String)
    Intake_Jurisdiction= Column(String)
    Intake_Date= Column(String)
    link= Column(String)


        
    def __repr__(self):
        return f"id={self.id}, name={self.Pet_ID}"

In [10]:
Base.metadata.create_all(engine)

In [19]:
metadata = MetaData(bind=engine)
conn = engine.connect()

In [11]:
engine.table_names()

['adoptable_dogs', 'dog_breed', 'lost_dogs']

In [12]:
found_data = adoption_df.to_dict(orient='records')

In [20]:
found_table = sqlalchemy.Table('adoptable_dogs', metadata, PrimaryKeyConstraint('id'),
                            autoload=True, extend_existing=True)

conn.execute(found_table.insert(), found_data)

In [21]:
def populate_table(engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)
    
# Call the function to insert the data for each table
populate_table(engine, Dog_breeds.__table__, 'Holly/dog_breed.csv')
populate_table(engine, Lost_Dogs.__table__, 'Holly/lost_pups.csv')
populate_table(engine, Adoptable_Dogs.__table__, 'Holly/adoptable_dogs.csv')


In [22]:
#run query to test data
engine.execute("SELECT * FROM dog_breed LIMIT 1").fetchall()

[(1, 'Affenpinscher', 5, 5, 1, 4, 1, 3, 2, 4, 3, 3, 4, 4, 1, 4, 3, 4, 4, 3, 3, 3, 1, 2, 1, 3, 3, 2)]

In [23]:
engine.execute("SELECT * FROM lost_dogs LIMIT 1").fetchall()

[(1, 'A1569847', 'M \xa0', '1\xa0Year \xa0', '47.0 LBS \xa0', 'BLACK WHITE \xa0', 'AM PIT BULL TER \xa0', 'ANAHEIM \xa0', '4/6/2018 \xa0')]

In [24]:
engine.execute("SELECT * FROM adoptable_dogs LIMIT 1").fetchall()

[(1, 'A1571111', 'F', '2 Months', '19.5 LBS', 'WHITE', 'AIREDALE TERR MIX', 'TUSTIN', '4/15/2018', 'http://petadoption.ocpetinfo.com/rescuetrack/servicethumb.php?tab=lost&detailid=A1571111')]